# Model Manager

In this notebook we prenset the [MlflowModelManager](../sheeprl/utils/model_manager.py) and a possible use.
It includes methods such as:
* Register the model
* Retrieve latest version
* Transition the model to a new stage
* Delete the model

First of all, we need to run Mlflow server with artifact store. You can find the instructions for running Mlflow server [here](https://mlflow.org/docs/latest/tracking.html#tracking-ui). Let's open a new terminal and run the following command:
```bash
mlflow ui
```

> **Note**
>
> This is one of the possibilities, you could have the server running on another machine, so you just need to set the `tracking_uri` parameter properly.

### Running the Experiment and Registering the Model
Second, we launch an experiment, so we need to retrieve the configs and execute the `run_algorithm` function. We train a PPO agent in the CartPole-v1 environment for few steps (we do not want to reach the best performance, but we want to show how SheepRL interprets model management for reinforcement learning).

In [1]:
import hydra
from omegaconf import OmegaConf
from sheeprl.utils.utils import dotdict
from sheeprl.cli import run_algorithm

# To retrieve the configs, we can simulate the cli command
# `python sheeprl.py exp=ppo total_steps=1024 model_manager.disabled=False logger@metric.logger=mlflow checkpoint.every=1024 exp_name=mlflow_example metric.logger.tracking_uri="http://localhost:5000"`
with hydra.initialize(version_base="1.3", config_path="../sheeprl/configs"):
    cfg = hydra.compose(
        config_name="config.yaml",
        overrides=[
            "exp=ppo",
            "total_steps=1024",
            "model_manager.disabled=False",
            "logger@metric.logger=mlflow",
            "checkpoint.every=1024",
            "exp_name=mlflow_example",
            "metric.logger.tracking_uri=http://localhost:5000",
        ],
    )
    cfg = dotdict(OmegaConf.to_container(cfg, resolve=True, throw_on_missing=True))
run_algorithm(cfg)

Seed set to 42
Experiment with name mlflow_example not found. Creating it.
/home/mmilesi/repos/sheeprl/sheeprl/utils/logger.py:79: UserWarning: Missing logger folder: logs/runs/ppo/CartPole-v1/2023-11-24_12-19-35_mlflow_example_42
  warnings.warn("Missing logger folder: %s" % save_dir, UserWarning)
/home/mmilesi/miniconda3/envs/sheeprl/lib/python3.10/site-packages/gymnasium/experimental/wrappers/rendering.py:166: UserWarning: WARN: Overwriting existing videos at /home/mmilesi/repos/sheeprl/examples/logs/runs/ppo/CartPole-v1/2023-11-24_12-19-35_mlflow_example_42/version_0/train_videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Encoder CNN keys: []
Encoder MLP keys: ['state']


/home/mmilesi/repos/sheeprl/sheeprl/algos/ppo/ppo.py:231: UserWarning: The metric.log_every parameter (5000) is not a multiple of the policy_steps_per_update value (512), so the metrics will be logged at the nearest greater multiple of the policy_steps_per_update value.
  warnings.warn(


Rank-0: policy_step=44, reward_env_1=11.0
Rank-0: policy_step=64, reward_env_3=16.0
Rank-0: policy_step=92, reward_env_2=23.0
Rank-0: policy_step=104, reward_env_3=10.0
Rank-0: policy_step=108, reward_env_0=27.0
Rank-0: policy_step=128, reward_env_1=21.0
Rank-0: policy_step=144, reward_env_2=13.0
Rank-0: policy_step=160, reward_env_0=13.0
Rank-0: policy_step=168, reward_env_1=10.0
Rank-0: policy_step=168, reward_env_3=16.0
Rank-0: policy_step=260, reward_env_1=23.0
Rank-0: policy_step=284, reward_env_0=31.0
Rank-0: policy_step=300, reward_env_2=39.0
Rank-0: policy_step=352, reward_env_1=23.0
Rank-0: policy_step=376, reward_env_0=23.0
Rank-0: policy_step=424, reward_env_0=12.0
Rank-0: policy_step=444, reward_env_1=23.0
Rank-0: policy_step=484, reward_env_0=15.0
Rank-0: policy_step=536, reward_env_1=23.0
Rank-0: policy_step=556, reward_env_3=97.0
Rank-0: policy_step=592, reward_env_2=73.0
Rank-0: policy_step=600, reward_env_1=16.0
Rank-0: policy_step=636, reward_env_0=38.0
Rank-0: policy

2023/11/24 12:19:48 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-10-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'sheeprl'}
/home/mmilesi/miniconda3/envs/sheeprl/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'mlflow_example'.
2023/11/24 12:19:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: mlflow_example, version 1


Registered model mlflow_example with version 1


Created version '1' of model 'mlflow_example'.


### Get Experiment Info

The experiment is logged on MLFlow, and we can retrieve it and the just  with the following instructions. Moreover, given the experiment it is possible to retrieve all the runs with the `mlflow.search_runs()` function.

> **Note**
>
> You can check these information also from a browser, by entering the MLFlow address in a browser, e.g., `http://localhost:5000` if you are running mlflow locally.

In [2]:
import mlflow

mlflow.set_tracking_uri(cfg.metric.logger.tracking_uri)
exp = mlflow.get_experiment_by_name("mlflow_example")
print("Experiment:", exp)
runs = mlflow.search_runs(experiment_ids=[exp.experiment_id])
print(f"Experiment ({exp.experiment_id}) runs:")
runs

Experiment: <Experiment: artifact_location='mlflow-artifacts:/541057322835557968', creation_time=1700824776648, experiment_id='541057322835557968', last_update_time=1700824776648, lifecycle_stage='active', name='mlflow_example', tags={}>
Experiment (541057322835557968) runs:


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.Time/sps_env_interaction,metrics.Loss/policy_loss,metrics.Loss/entropy_loss,metrics.Loss/value_loss,...,params.algo/clip_coef,params.exp_name,params.env/max_episode_steps,params.metric/aggregator/metrics/Loss/entropy_loss/sync_on_compute,params.metric/aggregator/metrics/Loss/value_loss/_target_,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.log-model.history
0,86ed7648208c47cbab275fa7e612e456,541057322835557968,FINISHED,mlflow-artifacts:/541057322835557968/86ed76482...,2023-11-24 11:19:36.855000+00:00,2023-11-24 11:19:48.951000+00:00,329.871509,-5.913153,-0.687031,36.166283,...,0.2,mlflow_example,None,False,torchmetrics.MeanMetric,mmilesi,ppo_CartPole-v1_2023-11-24_12-19-35,LOCAL,/home/mmilesi/miniconda3/envs/sheeprl/lib/pyth...,"[{""run_id"": ""86ed7648208c47cbab275fa7e612e456""..."


### Retrieve Model Info
Since we set the `model_manager.disabled` to `False` the PPO Agent is registered in MLFLow, we can get its information with the following instructions.

In [3]:
from sheeprl.utils.model_manager import MlflowModelManager
from lightning import Fabric

fabric = Fabric(devices=1, accelerator=cfg.fabric.accelerator, precision=cfg.fabric.precision)
fabric.launch()
model_manager = MlflowModelManager(fabric, cfg.model_manager.tracking_uri)

model_info = mlflow.search_registered_models(filter_string="name='mlflow_example'")[-1]
model_name = model_info.name
print("Name:", model_name)
print("Description:", model_info.description)
print("Tags:", model_info.tags)
latest_version = model_manager.get_latest_version(model_info.name)
print("Latest Version:", latest_version.version)

Name: mlflow_example
Description: # MODEL CHANGELOG
## **Version 1**
### Author: mmilesi
### Date: 24/11/2023 12:19:48 CET
### Description: 
PPO Agent in CartPole-v1 Environment

Tags: {}
Latest Version: 1


### Registering a New Model Version from Checkpoint

Suppose to train a new PPO Agent in the CartPole-v1 environment and to obtain better results than before. You can register a new version of the model. To do this, we show another method to register models, not directly after training, but from a checkpoint.

First of all, we need to run another experiment with different hyper-parameters.

In [4]:
# To retrieve the configs, we can simulate the cli command
# `python sheeprl.py exp=ppo total_steps=16384 checkpoint.every=16384 logger@metric.logger=mlflow exp_name=mlflow_example metric.logger.tracking_uri="http://localhost:5000"`
import os

with hydra.initialize(version_base="1.3", config_path="../sheeprl/configs"):
    cfg_ = hydra.compose(
        config_name="config.yaml",
        overrides=[
            "exp=ppo",
            "total_steps=16384",
            "checkpoint.every=16384",
            "logger@metric.logger=mlflow",
            "exp_name=mlflow_example",
            "metric.logger.tracking_uri=http://localhost:5000",
        ],
    )
    cfg = dotdict(OmegaConf.to_container(cfg_, resolve=True, throw_on_missing=True))
run_algorithm(cfg)
os.mkdir(f"./logs/runs/{cfg.root_dir}/{cfg.run_name}/.hydra/")
OmegaConf.save(cfg_, f"./logs/runs/{cfg.root_dir}/{cfg.run_name}/.hydra/config.yaml")

Seed set to 42
/home/mmilesi/repos/sheeprl/sheeprl/utils/logger.py:79: UserWarning: Missing logger folder: logs/runs/ppo/CartPole-v1/2023-11-24_12-19-49_mlflow_example_42
  warnings.warn("Missing logger folder: %s" % save_dir, UserWarning)
/home/mmilesi/miniconda3/envs/sheeprl/lib/python3.10/site-packages/gymnasium/experimental/wrappers/rendering.py:166: UserWarning: WARN: Overwriting existing videos at /home/mmilesi/repos/sheeprl/examples/logs/runs/ppo/CartPole-v1/2023-11-24_12-19-49_mlflow_example_42/version_0/train_videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/home/mmilesi/repos/sheeprl/sheeprl/algos/ppo/ppo.py:231: UserWarning: The metric.log_every parameter (5000) is not a multiple of the policy_steps_per_update value (512), so the metrics will be logged at the nearest greater multiple of the policy_steps_per_update value.
  warnings.warn(


Encoder CNN keys: []
Encoder MLP keys: ['state']
Rank-0: policy_step=44, reward_env_1=11.0
Rank-0: policy_step=64, reward_env_3=16.0
Rank-0: policy_step=92, reward_env_2=23.0
Rank-0: policy_step=104, reward_env_3=10.0
Rank-0: policy_step=108, reward_env_0=27.0
Rank-0: policy_step=128, reward_env_1=21.0
Rank-0: policy_step=144, reward_env_2=13.0
Rank-0: policy_step=160, reward_env_0=13.0
Rank-0: policy_step=168, reward_env_1=10.0
Rank-0: policy_step=168, reward_env_3=16.0
Rank-0: policy_step=260, reward_env_1=23.0
Rank-0: policy_step=284, reward_env_0=31.0
Rank-0: policy_step=300, reward_env_2=39.0
Rank-0: policy_step=352, reward_env_1=23.0
Rank-0: policy_step=376, reward_env_0=23.0
Rank-0: policy_step=424, reward_env_0=12.0
Rank-0: policy_step=444, reward_env_1=23.0
Rank-0: policy_step=484, reward_env_0=15.0
Rank-0: policy_step=536, reward_env_1=23.0
Rank-0: policy_step=556, reward_env_3=97.0
Rank-0: policy_step=592, reward_env_2=73.0
Rank-0: policy_step=600, reward_env_1=16.0
Rank-0: 

Now we can use the `./sheeprl_model_manager.py` script to take a checkpoint and register the models of the checkpoint.

In [5]:
from sheeprl.cli import registration

# To retrieve the configs, we can simulate the cli command
# `python sheeprl_model_manager.py checkpoint_path=/path/to/checkpoint.ckpt \
# model_manager=ppo model_manager.models.agent.description='New PPO Agent version trained in CartPole-v1 environment'`
with hydra.initialize(version_base="1.3", config_path="../sheeprl/configs"):
    cfg = hydra.compose(
        config_name="model_manager_config.yaml",
        overrides=[
            # Substitute the checkpoint path with your /path/to/checkpoint.ckpt
            "checkpoint_path=./logs/runs/ppo/CartPole-v1/2023-11-24_12-19-49_mlflow_example_42/version_0/checkpoint/ckpt_16384_0.ckpt",
            "model_manager=ppo",
            "model_manager.models.agent.description='New PPO Agent version trained in CartPole-v1 environment'",
        ],
    )
registration(cfg)

2023/11/24 12:20:40 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-10-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'sheeprl'}
Registered model 'mlflow_example' already exists. Creating a new version of this model...
2023/11/24 12:20:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: mlflow_example, version 2


Registered model mlflow_example with version 2


Created version '2' of model 'mlflow_example'.


And, of course, we can retrieve the new information of the registred model.

In [6]:
model_info = mlflow.search_registered_models(filter_string=f"name='{model_name}'")[-1]
print("Name:", model_info.name)
print("Description:", model_info.description)
print("Tags:", model_info.tags)
latest_version = model_manager.get_latest_version(model_info.name)
print("Latest Version:", latest_version.version)

Name: mlflow_example
Description: # MODEL CHANGELOG
## **Version 1**
### Author: mmilesi
### Date: 24/11/2023 12:19:48 CET
### Description: 
PPO Agent in CartPole-v1 Environment
## **Version 2**
### Author: mmilesi
### Date: 24/11/2023 12:20:40 CET
### Description: 
New PPO Agent version trained in CartPole-v1 environment

Tags: {}
Latest Version: 2


### Staging the Model
After registering the model, we can transition the model to a new stage. We can transition the model to staging stage with the following command.

In [7]:
model_manager.transition_model(
    model_name="mlflow_example", version=latest_version.version, stage="staging", description="Staging Model for demo"
)

Transitioning model mlflow_example version 2 from None to staging


<ModelVersion: aliases=[], creation_timestamp=1700824840570, current_stage='Staging', description=('# MODEL CHANGELOG\n'
 '## **Version 2**\n'
 '### Author: mmilesi\n'
 '### Date: 24/11/2023 12:20:40 CET\n'
 '### Description: \n'
 'New PPO Agent version trained in CartPole-v1 environment\n'), last_updated_timestamp=1700824844145, name='mlflow_example', run_id='1ecf72c9e0dd43a68e13c19919b7258b', run_link='', source='mlflow-artifacts:/765086633400039374/1ecf72c9e0dd43a68e13c19919b7258b/artifacts/agent', status='READY', status_message='', tags={}, user_id='', version='2'>

### Downloading the Model
You can download the registered models and load with the `torch.load()` function.

In [8]:
import torch

download_path = "./models/ppo-agent-cartpole"
model_manager.download_model(model_name, latest_version.version, download_path)
agent = torch.load("models/ppo-agent-cartpole/agent/data/model.pth")
agent

/home/mmilesi/miniconda3/envs/sheeprl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating output path ./models/ppo-agent-cartpole


PPOAgent(
  (feature_extractor): MultiEncoder(
    (mlp_encoder): MLPEncoder(
      (model): MLP(
        (_model): Sequential(
          (0): Linear(in_features=4, out_features=64, bias=True)
          (1): Tanh()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Tanh()
          (4): Linear(in_features=64, out_features=64, bias=True)
        )
      )
    )
  )
  (critic): MLP(
    (_model): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
      (4): Linear(in_features=64, out_features=1, bias=True)
    )
  )
  (actor_backbone): MLP(
    (_model): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (actor_heads): ModuleList(
    (0): Linear(in_features=64, out_features=2, bias=True)
  )
)

### Delete Model
Finally, you can delete registered models you no longer need.

In [9]:
model_manager.delete_model(
    model_name, int(latest_version.version) - 1, f"Delete model version {int(latest_version.version)-1}"
)
mlflow.search_registered_models(filter_string="name='mlflow_example'")[-1]

Deleting model mlflow_example version 1


<RegisteredModel: aliases={}, creation_timestamp=1700824788962, description=('# MODEL CHANGELOG\n'
 '## **Version 1**\n'
 '### Author: mmilesi\n'
 '### Date: 24/11/2023 12:19:48 CET\n'
 '### Description: \n'
 'PPO Agent in CartPole-v1 Environment\n'
 '## **Version 2**\n'
 '### Author: mmilesi\n'
 '### Date: 24/11/2023 12:20:40 CET\n'
 '### Description: \n'
 'New PPO Agent version trained in CartPole-v1 environment\n'
 '## **Transition:**\n'
 '### Version 2 from None to Staging\n'
 '### Author: mmilesi\n'
 '### Date: 24/11/2023 12:20:44 CET\n'
 '### Description: \n'
 'Staging Model for demo\n'
 '## **Deletion:**\n'
 '### Version 1 from stage: None\n'
 '### Author: mmilesi\n'
 '### Date: 24/11/2023 12:21:01 CET\n'
 '### Description: \n'
 'Delete model version 1\n'), last_updated_timestamp=1700824861631, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1700824840570, current_stage='Staging', description=('# MODEL CHANGELOG\n'
 '## **Version 2**\n'
 '### Author: mmilesi\n'
 '